In [2]:
from wayne_utils import load_data, save_data
import os
import numpy as np
import pandas as pd

_ROOT_PATH = "/home/jiangpeiwen2/jiangpeiwen2/projects/TKGT/T-Tuple-T"

label_count_list = load_data( os.path.join( _ROOT_PATH, "label_count_list_test.json"), "json")

## 3 从预测结果构建统计数据

In [5]:
def flatten( count_list ):
    save_list = []
    for i in range( len(count_list)):
        temp_dict = {}
        try:
            labels = count_list[str(i)]
        except:
            labels = count_list[ i ]
        for key in ['法院', '原告', '被告']:
            temp_dict[key] = labels[key]
        for entity in ['法院', '原告']:
            for key in labels["事实"][entity]:
                prefix = "原告诉称" if entity == "原告" else "法院裁定"
                temp_dict[ f"{prefix}_{key}"] = labels["事实"][entity][key]
        save_list.append( temp_dict )
    return save_list

def count_from_predict( predict_dict ):
    empty_dict = {
        '法院': 0,
        '原告': 0,
        '被告': 0,
        '事实': {
            '原告': {
                '借款凭证': 0,
                '约定的借款金额': 0,
                '约定的还款日期或借款期限': 0,
                '约定的利息': 0,
                '约定的逾期利息': 0,
                '约定的违约金': 0
            },
            '法院': {
                '借款凭证': 0,
                '约定的借款金额': 0,
                '约定的还款日期或借款期限': 0,
                '约定的利息': 0,
                '约定的逾期利息': 0,
                '约定的违约金': 0
            }
        }
    }
    for item in predict_dict:
        entity_type, field, value = item[0], item[1], item[2]
        if entity_type not in ["原告", "被告", "法院"]:
            continue
        elif entity_type == "被告" and field != "姓名名称":
            continue
        if field == "姓名名称":
            empty_dict[ entity_type ] += 1
        else:
            for key in ['借款凭证', '约定的借款金额', '约定的还款日期或借款期限', '约定的利息', '约定的逾期利息', '约定的违约金']:
                if key in field:
                    try:
                        empty_dict[ '事实' ][ entity_type ][key] += 1
                    except:
                        raise Exception( item )
    return empty_dict

In [11]:

model_list_table = [
    "ChatGLM3-6B",
    "Qwen1.5-7B-Chat",
    "Baichuan2-7B-Chat",
    "Chinese-Mistral-7B-Instruct-v0.1",
    "Qwen2.5-0.5B",

    "CPL_dynamic_tabel_TTT_ChatGLM3-6B-5epoch",
    "CPL_dynamic_tabel_TTT_Qwen1.5-7B-Chat-5epoch",
    "CPL_dynamic_tabel_Chinese-Mistral-7B-Instruct-v0.1-5epoch",
]
scopes = model_list_table
scopes

['ChatGLM3-6B',
 'Qwen1.5-7B-Chat',
 'Baichuan2-7B-Chat',
 'Chinese-Mistral-7B-Instruct-v0.1',
 'Qwen2.5-0.5B',
 'CPL_dynamic_tabel_TTT_ChatGLM3-6B-5epoch',
 'CPL_dynamic_tabel_TTT_Qwen1.5-7B-Chat-5epoch',
 'CPL_dynamic_tabel_Chinese-Mistral-7B-Instruct-v0.1-5epoch']

In [12]:
import sys
sys.path.insert( 0, "/home/jiangpeiwen2/jiangpeiwen2/projects/TKGT/evaluation/CountCPL")
from evaluate import Eval
for model_name in scopes:
    predict_lists = load_data( os.path.join( _ROOT_PATH, f"CPL_predict_list_filter_{model_name}.pickle"), "pickle")
    predict_count_list = []
    for index in range( len(predict_lists)):
        ret_dict = count_from_predict( predict_lists[ index ] )
        predict_count_list.append( ret_dict )
    predict_count_list = flatten( predict_count_list )
    aadf, error_rate = Eval( predict_count_list, label_count_list )
    print( model_name )
    print( aadf.describe().loc["mean"])


test 211 tables for sss
Error rate = 98.89415481832543 20.458135860979464 69.11532385466035 55.60821484992101
ChatGLM3-6B
Easy-RMSE       0.887919
Easy-MAE        0.594787
Easy-EM        99.447077
Medium-RMSE     0.277096
Medium-MAE      0.117299
Medium-EM      60.229068
Hard-RMSE       0.765628
Hard-MAE        0.445498
Hard-EM        84.557662
AVG-RMSE        0.691163
AVG-MAE         0.344076
AVG-EM         77.804107
Name: mean, dtype: float64
test 211 tables for sss
Error rate = 95.41864139020537 26.619273301737756 69.03633491311217 57.345971563981045
Qwen1.5-7B-Chat
Easy-RMSE       0.859426
Easy-MAE        0.569510
Easy-EM        97.709321
Medium-RMSE     0.428366
Medium-MAE      0.202607
Medium-EM      63.309637
Hard-RMSE       0.741840
Hard-MAE        0.433254
Hard-EM        84.518167
AVG-RMSE        0.724397
AVG-MAE         0.368246
AVG-EM         78.672986
Name: mean, dtype: float64
test 211 tables for sss
Error rate = 100.0 19.747235387045812 68.95734597156398 55.48183254344391